In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
import os
import warnings
warnings.filterwarnings("ignore")
import os
for file in os.listdir('../input/'):
    print(file.ljust(30)+str(round(os.path.getsize('../input/'+file)/1000000,2))+'MB')

In [ ]:
ckpt = datetime.now()
train_data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv',index_col='test_id')
print(f'time cost: {datetime.now() - ckpt}')

In [ ]:
print(len(train_data[train_data['is_duplicate']==1]))
print(len(train_data[train_data['is_duplicate']==0]))
print('The ratio:',len(train_data[train_data['is_duplicate']==1])/len(train_data))

# naive guess for benchmark

In [ ]:
# from sklearn.metrics import log_loss
# probability = train_data['is_duplicate'].mean()
# naive_loss = log_loss(train_data['is_duplicate'].astype('float'),np.zeros((len(train_data),1))+probability)
# print(naive_loss)
# print(train_data[train_data['is_duplicate']==1][['question1', 'question2']].head(5))
# print(train_data[train_data['is_duplicate']==0][['question1', 'question2']].head(5))

# NLP feature enginnering


Let's try embedding * tf-idf  seems good.


[What does word embedding weighted by tf-idf mean?](https://stats.stackexchange.com/questions/318910/what-does-word-embedding-weighted-by-tf-idf-mean)

[Word2Vec embeddings with TF-IDF](https://datascience.stackexchange.com/questions/28598/word2vec-embeddings-with-tf-idf)

[What does a weighted word embedding mean?](https://stackoverflow.com/questions/47727078/what-does-a-weighted-word-embedding-mean)

[NILC-USP at SemEval-2017 Task 4: A Multi-view Ensemble for Twitter Sentiment Analysis](http://www.aclweb.org/anthology/S17-2100)
> In this work, tweets were modeled using three types of text representation. The first one is a bag-of-words model weighted by tf-idf (term frequency - inverse document frequency) (Section 2.1.1). The second represents a sentence by averaging the word embeddings of all words (in the sentence) and the third represents a sentence by averaging the weighted word embeddings of all words, the weight of a word is given by tf-idf (Section 2.1.2).

In [ ]:
import psutil
psutil.virtual_memory()

In [ ]:
print("Let's check Nan before processing for train data")
from IPython.display import display, HTML
print('In training data')
print('\nquestion1 is nan:')
display(train_data[train_data['question1'].apply(lambda x:x is np.nan)])
print('question2 is nan:')
display(train_data[train_data['question2'].apply(lambda x:x is np.nan)])


print("Let's check Nan before processing for test data")
print('In testing data')
print('\nquestion1 is nan:')
display(test_data[test_data['question1'].apply(lambda x:x is np.nan)])
print('question2 is nan:')
display(test_data[test_data['question2'].apply(lambda x:x is np.nan)])

print('special case')
display(test_data[(test_data['question1'].apply(lambda x:x is np.nan)) & (test_data['question2'].apply(lambda x:x is np.nan))])
# test_data.loc[['life in dublin?"'],:]

print('Give 0 to these test rows in default.')

Legacy code, since we have to reserved  data for submission.

        def delete_nan(data):
            print('shape before delete',data.shape)
            data.drop(data[data['question1'].apply(lambda x:x is np.nan)].index,axis=0,inplace=True)
            data.drop(data[data['question2'].apply(lambda x:x is np.nan)].index,axis=0,inplace=True)
            print('shape after delete',data.shape)
            return data
         
        train_data = delete_nan(train_data)
        test_data = delete_nan(test_data)
        corpus = []
        for data in [train_data,test_data]:
            for column in ['question1','question2']:
                corpus+=list(data[column])

Keep them now. Give 0 to these test rows in default.

In [ ]:
psutil.virtual_memory()

In [ ]:
corpus =list(train_data['question1'])+list(train_data['question2'])
print('the nan: ',list(filter(lambda x:x[0] if x[1] is np.nan else None, enumerate(corpus))))
corpus = list(map(lambda x:'' if x is np.nan else x, corpus))

In [ ]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from gensim.models import FastText
corpus_tokenized = [nltk.word_tokenize(str(sent)) for sent in corpus]
word_embedding_model = FastText(corpus_tokenized,window=5, min_count=0,workers=1000)
word_embedding_model.save('./fasttext.model')

# I don't think remove stopwords before embedding is a good idea, stopwords often show relation between entities, which is important for context, but you can still try it.

```
'What is the step by step guide to invest in share market in india?',
 'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
 'How can I increase the speed of my internet connection while using a VPN?',
 'Why am I mentally very lonely? How can I solve it?',
 'Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?'
 ```
 
```
[['What', 'step', 'step', 'guide', 'invest', 'share', 'market', 'india', '?'], 
['What', 'story', 'Kohinoor', '(', 'Koh-i-Noor', ')', 'Diamond', '?'], 
['How', 'I', 'increase', 'speed', 'internet', 'connection', 'using', 'VPN', '?'], 
['Why', 'I', 'mentally', 'lonely', '?', 'How', 'I', 'solve', '?'], 
['Which', 'one', 'dissolve', 'water', 'quikly', 'sugar', ',', 'salt', ',', 'methane', 'carbon', 'di', 'oxide', '?'],
 ```

In [ ]:
def embedding_after_filter_stopwords():
    for idx,sent in enumerate(corpus):
        corpus[idx] = list(filter(lambda word: word not in stop_words,sent))
    word_embedding_model = FastText(corpus,window=5, min_count=1)
    word_embedding_model.save('./fasttext_with_stopwords.model')
    return word_embedding_model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
ckpt = datetime.now()
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
print(f'Time cost: {datetime.now()-ckpt}')

In [ ]:
train_data[train_data['question1'].apply(lambda x:x is np.nan)]

In [ ]:
filter_npnan(train_data.loc[363362,'question1'])

In [ ]:
ckpt = datetime.now()
def filter_npnan(x):
    return '' if x is np.nan else x
x1 = train_data['question1'].apply(filter_npnan)
x2 = train_data['question2'].apply(filter_npnan)
print('Filtered')
x1 = vectorizer.transform(x1)
x2 = vectorizer.transform(x2)
print(f'Time cost: {datetime.now()-ckpt}')
print(type(x1))
print(type(x2))

In [ ]:
y = train_data['is_duplicate']


[Compressed Sparse Row matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)


In [ ]:
print('The size of bag of words after tf-ifd',len(vectorizer.get_feature_names()))
print('corpus length:',len(corpus))
print('Shape:',x1.shape)
print('Shape:',x2.shape)

In [ ]:
print('type of tfidf vocab',type(vectorizer.vocabulary_))
print('length of the vocab',len(vectorizer.vocabulary_))

In [ ]:
from scipy.sparse import hstack
x = hstack([x1,x2])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [ ]:
est = RandomForestRegressor()
est.fit(x_train,y_train)
print(est.score(x_test, y_test))

In [ ]:
# x_train = np.random.randn(100,6)
# y_train = x_train[:,0]*5+x_train[:,1]*2+x_train[:,3]*4

In [ ]:
from catboost import Pool, CatBoostRegressor, cv
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=5,
    random_seed=0,
    learning_rate=0.1
)
model.fit(
   x_train, y_train,
    logging_level='Verbose',
    plot=True
);

In [ ]:
import tensorflow as tf